# Adhikari Dataset Conversion

This program edits the dataset for the Adhikari Fall Detection Dataset

In [ ]:
# google drive related
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

# csv related
from google.colab import files

Mounted at /content/drive


------------------------------------------------------
Code below related to google drive

In [ ]:
# Only need to edit the 'dataset' path
shared_directory = '/content/drive/Shareddrives/Team56_FallDetection'     # Shared drive path
dataset_directory = 'AdhikariSet/1301'                                    # Edit the dataset path
full_directory = shared_directory + '/' +  dataset_directory
full_path = full_directory + '/labels.csv'   # string Full path


--------------------------
Accessing CSV

```
**Adhikari Labels**
0 = Empty
1 = Standing
2 = Sitting
3 = Lying/Fall
4 = Bending
5 = Crawling
6 = Empty?
```

```
**Dataset Label Conversion**
          Fall (1)  = 3
Temporary Pose (0)  = 4 5
      Not Fall (-1) = 0 1 2 6
```

In [ ]:
# Read path, create dataframe object
df = pd.read_csv(full_path)

# replace original label with temporary value
df.loc[df['class'] == 0, 'class'] = 'not'
df.loc[df['class'] == 1, 'class'] = 'not'
df.loc[df['class'] == 2, 'class'] = 'not'
df.loc[df['class'] == 3, 'class'] = 'fall'
df.loc[df['class'] == 4, 'class'] = 'temporary'
df.loc[df['class'] == 5, 'class'] = 'temporary'
df.loc[df['class'] == 6, 'class'] = 'not'

# replace with numerical value
df.loc[df['class'] == 'not', 'class'] = -1
df.loc[df['class'] == 'temporary', 'class'] = 0
df.loc[df['class'] == 'fall', 'class'] = 1

temp_path = full_directory + '/test.csv'
df.to_csv(temp_path)

# Images to MP4

Libraries

In [ ]:
# images to mp4
import os
import cv2
from PIL import Image

# connect to drive
from google.colab import drive
drive.mount('/content/drive')

# resize images
import os.path
from os import path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---
Variable List (exclude temporaries) <br><br>


Path to directory
*   full_path
<br><br>

Set Video Size
*   list1
*   num_images
*   mean_width
*   mean_height
<br><br>

Resize Images
*   resize_path
<br><br>

Video Generation
*   output_path
*   video_name
*   images
*   video
<br><br>



---

Path to directory

In [ ]:
# Edit location of images
#image_directory = 'fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/fall-01-cam0-rgb'                                    # Edit the dataset path

# Do not edit
#shared_directory = '/content/drive/Shareddrives/Team56_FallDetection'     # Shared drive path
#full_path = shared_directory + '/' +  image_directory

full_path = "/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/fall-02-cam0-rgb_labelled"

---
Set Video Size from average image size

In [ ]:
# make list of all files in directory
list1=os.listdir(full_path)
list1.sort()


# get number of images in directory
num_images = len(list1)
print(num_images)


# get average size of all images in list
mean_width = 0
mean_height = 0

for file in list1:
  #print(file)
  im = Image.open(os.path.join(full_path, file))
  width, height = im.size
  mean_width += width
  mean_height += height
  im.show()

mean_width = int(mean_width / num_images)
mean_height = int(mean_height / num_images)
print(mean_width, mean_height)

108
640 480


---
Resize images to fit video

In [ ]:
# makes directory for the resized images
resize_path = '/content/resized_images'
if not path.isdir(resize_path):
  print("Creating new directory")
  os.mkdir(resize_path)
else:
  print("Removing old content in directory")
  for f in os.listdir(resize_path):
    os.remove(os.path.join(resize_path, f))



# resize images
for file in list1:
  # get current size of image
  im = Image.open(os.path.join(full_path, file))
  width, height = im.size
  #print(width, height)

  # resize
  imResize = im.resize((mean_width, mean_height), Image.ANTIALIAS)

  resize_filename = resize_path + "/resize_" + file
  imResize.save(resize_filename, 'JPEG', quality = 95)
  #print(im.filename.split('\\')[-1], " is resized")

print("Success")

Removing old content in directory
Success


---
Video Generation

In [ ]:
# creates directory for output video
output_path = '/content/output_videos'
if not path.isdir(output_path):
  print("Creating new directory")
  os.mkdir(output_path)


# video name
video_name = path.basename(full_path) + ".avi"
#print(video_name)

# get image list
images = os.listdir(resize_path)
images.sort()
#print(images)


# set frame settings to first image
print("creating video")
frame = cv2.imread(os.path.join(resize_path, images[0]))
height, width, layers = frame.shape
save_location = output_path + '/' + video_name
video = cv2.VideoWriter(save_location, 0, 30, (width, height))
if path.isfile(save_location):
  print("video created")

# append images
for image in images:
  video.write(cv2.imread(os.path.join(resize_path, image)))

# deallocate memory
cv2.destroyAllWindows()
video.release()
print("success")

creating video
video created
success


In [ ]:
print("Removing old content in directory")
for f in os.listdir(resize_path):
  os.remove(os.path.join(resize_path, f))

Removing old content in directory
